In [1]:
import os, sys, re, pickle
import pandas as pd
import numpy as np

from tqdm import tqdm

from collections import Counter

import os.path as osp

datafolderpath = "../dataset"
test_csv = "test - no labels.csv"
test_human_pkl = "test_human_generated.pkl"

testdf_file_path = "/home/prashantk/INLG-shared-task/PreCogLTRC-HinglishEval-INLG-2022/dataset/testdf.json"

In [2]:
testdf = pd.read_json(testdf_file_path)

In [3]:
testdf.columns

Index(['index', 'English', 'Hindi', 'Hinglish', 'hum_gen', 'Hinglish_csnliop',
       'hum_gen_csnliop', 'hum_gen_norm', 'hum_gen_tokens', 'hum_gen_lid',
       'Hinglish_norm', 'Hinglish_lid', 'Hinglish_pos', 'hum_gen_pos',
       'Hinglish_cmi_scores', 'Hinglish_burstiness_scores',
       'Hinglish_sp_scores', 'symcom_scores_pos', 'symcom_scores_sent',
       'NOUN_symcom', 'PART_symcom', 'PRON_symcom', 'ADP_symcom', 'ADJ_symcom',
       'ADV_symcom', 'VERB_symcom', 'AUX_symcom', 'CCONJ_symcom',
       'SCONJ_symcom', 'DET_symcom', 'PROPN_symcom', 'NUM_symcom',
       'INTJ_symcom', 'symcom_+1_count', 'symcom_-1_count', 'symcom_na_count',
       'symcom_others', 'Hingilish_xlmr_scores', 'hum_gen_xlmr_scores'],
      dtype='object')

In [10]:
testdf = pd.read_csv(osp.join(datafolderpath, test_csv))
test_hum_gen_map = pickle.load(open(osp.join(datafolderpath, test_human_pkl), 'rb'))
hum_generated_column = []

for ind, row in testdf.iterrows():
    try:
        hum_generated_column.append(test_hum_gen_map[row["English"]])
    except:
        print(row["English"])
        query="'"+row["English"]
        hum_generated_column.append(test_hum_gen_map[query])
testdf["hum_gen"] = hum_generated_column

testdf.shape

In exercise of the powers vested in him by Rule 58 1 paragraph b of the Defence of Hyderabad Rules which correspond to the Defence of India Rules, it is hereby ordered that no person shall unfurl in a ceremonial fashion at any public function the flag of any foreign country nor salute it.



(791, 4)

# csnli


In [13]:
os.chdir('/home/prashantk/csnli')

print(os.getcwd())
from three_step_decoding import *


tsd = ThreeStepDecoding('/home/prashantk/csnli/lid_models/hinglish', 
                        htrans='/home/prashantk/csnli/nmt_models/rom2hin.pt', 
                        etrans='/home/prashantk/csnli/nmt_models/eng2eng.pt')

#testing if csnli tool is working
print('\n'.join(['\t'.join(x) for x in tsd.tag_sent(u'i thght mosam dfrnt hoga bs fog h')]))

/home/prashantk/csnli
Loading NMT model parameters.
Loading NMT model parameters.
The dy.parameter(...) call is now DEPRECATED.
        There is no longer need to explicitly add parameters to the computation graph.
        Any used parameter will be added automatically.
i	i	en
thght	thought	en
mosam	मौसम	hi
dfrnt	different	en
hoga	होगा	hi
bs	बस	hi
fog	fog	en
h	है	hi


In [14]:
def normalise_sent(sent):
    try:
        ret = list(tsd.tag_sent(sent))
    except TypeError:
        ret = [sent]
        print(f"TypeError : {sent}")
    return ret
              
def normalise_df_column(df, column_name):
    csnli_op = []
    if isinstance(df[column_name][0], str):
        csnli_op = [normalise_sent(sent) for sent in tqdm(df[column_name])]
        
    elif isinstance(df[column_name][0], list):
        csnli_op = [[normalise_sent(sent) for sent in sample] for sample in tqdm(df[column_name]) ]
        
    return csnli_op

columns_to_normalise = ["Hinglish", "hum_gen"]


    
for col in columns_to_normalise:
    testdf[col+"_csnliop"] = normalise_df_column(testdf, col)

    
tokens_all,norm_tokens_all, lid_all = [],[],[]

for row in testdf["hum_gen_csnliop"]:
    
    row_tokens, row_norm_tokens, row_lid = [],[],[]
    
    for sent in row:
        tokens = [token[0] for token in sent]
        norm_tokens = [token[1] for token in sent]
        lid = [token[2] for token in sent]
        
        row_tokens.append(tokens)
        row_norm_tokens.append(norm_tokens)
        row_lid.append(lid)
        
    tokens_all.append(row_tokens)
    norm_tokens_all.append(row_norm_tokens)
    lid_all.append(row_lid)

testdf["hum_gen_norm"] = norm_tokens_all
testdf["hum_gen_tokens"] = tokens_all
testdf["hum_gen_lid"] = lid_all   

100%|██████████| 791/791 [04:14<00:00,  3.11it/s]


In [6]:
testdf.columns

Index(['English', 'Hindi', 'Hinglish', 'hum_gen', 'Hinglish_csnliop',
       'hum_gen_csnliop', 'hum_gen_norm', 'hum_gen_tokens', 'hum_gen_lid'],
      dtype='object')

In [ ]:
for row in testdf["Hinglish_csnliop"]:
    
    row_tokens, row_norm_tokens, row_lid = [],[],[]
    
    for sent in row:
        tokens = [token[0] for token in sent]
        norm_tokens = [token[1] for token in sent]
        lid = [token[2] for token in sent]
        
        row_tokens.append(tokens)
        row_norm_tokens.append(norm_tokens)
        row_lid.append(lid)
        
    tokens_all.append(row_tokens)
    norm_tokens_all.append(row_norm_tokens)
    lid_all.append(row_lid)

testdf["hum_gen_norm"] = norm_tokens_all
testdf["hum_gen_tokens"] = tokens_all
testdf["hum_gen_lid"] = lid_all   

In [7]:
testdf["Hinglish_norm"] = [[token[1] for token in el] for el in testdf["Hinglish_csnliop"]]
testdf["Hinglish_lid"] = [[token[2] for token in el] for el in testdf["Hinglish_csnliop"]]

In [15]:
testdf.to_json(testdf_file_path)

# PoS Tagging

In [4]:
testdf = pd.read_json(testdf_file_path)

In [8]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from datasets import load_dataset, load_metric, concatenate_datasets


tokenizer = AutoTokenizer.from_pretrained("prakod/en-hi-pos-tagger-symcom")

model = AutoModelForTokenClassification.from_pretrained("prakod/en-hi-pos-tagger-symcom")


def predictposSent(model,sentence):
  
  tokenized_sentence = tokenizer(sentence,return_tensors='pt')


  mask = []
  prev_id = None
  for ind,id in enumerate(tokenized_sentence.word_ids()):
    
    if id is None:
      mask.append(-100)
    elif id == prev_id:
      mask.append(-100)
    elif id != prev_id:
      mask.append(id)
    prev_id = id


  outputs = model(**tokenized_sentence.to('cuda'))

  preds = np.argmax(outputs['logits'].cpu().detach().numpy(), axis=2).squeeze()


  true_preds = [
      label_list[p] for (p, l) in zip(preds, mask) if l != -100
  ]
  
  
  return true_preds


sent = ['abbe','keyo', 'fek', 'raha', 'hai', 'andhe', 'news', 'dalal'] 
        
sent_norm = ['अब्बी', 'क्यो', 'फेक', 'रहा', 'है', 'अंधे', 'news', 'डाल' ]

model.to('cuda')

datasets_UD = load_dataset('/home/prashantk/en-hi-pos-tagger/load_UD_enhics_mod (3).py','qhe_hiencs')

label_list = datasets_UD["test"].features[f"upos"].feature.names

tags_words = predictposSent(model,' '.join(sent))
tags_normalised = predictposSent(model,' '.join(sent_norm))

testdf["Hinglish_pos"] = [predictposSent(model,' '.join(sent)) for sent in tqdm(testdf["Hinglish_norm"])]
testdf["hum_gen_pos"] = [[predictposSent(model,' '.join(sent)) for sent in el] for el in tqdm(testdf["hum_gen_norm"])]

Reusing dataset universal_dependencies (/home/prashantk/.cache/huggingface/datasets/universal_dependencies/qhe_hiencs/0.0.0/8a3dbde569b161527eabcc186381ac424d4859ff64505fb3da8ac3e0a03c904d)


  0%|          | 0/3 [00:00<?, ?it/s]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 791/791 [00:22<00:00, 35.14it/s]


# code-mix metrics

In [9]:
sys.path.append('/home/prashantk/cs_metrics')

from cs_metrics import *

sample = 'EN EN HI HI UNIV UNIV HI HI EN EN EN HI HI'
cmi(sample)


45.45454545454546

In [10]:
cmi_scores = []

for ind, row in tqdm(testdf.iterrows()):
    
    lid = [l.upper() for l in row["Hinglish_lid"]]
    
    try:
        cmi_score = cmi(' '.join(lid))
    
    except IndexError as e:
        print(lid,e)
        cmi_score = 0
    
    cmi_scores.append(cmi_score)

    
testdf['Hinglish_cmi_scores'] = cmi_scores

def spavg_test(x, k= 2):
        
    LANG_TAGS = ['EN', 'HI']
    OTHER_TAGS = ['UNIV', 'NE','ACRO']

    LANG_TAGS = [tag.lower() for tag in LANG_TAGS]
    OTHER_TAGS = [tag.lower() for tag in OTHER_TAGS]

    if isinstance(x,str):
        x = x.split()
        
    x = [el.lower() for el in x]

    count = 0 
    mem = None
    for l_i, l_j in zip(x,x[1:]):
        if l_i in OTHER_TAGS:
            continue
        if l_i != l_j:
            count+=1

    return count 


sp_scores = []

for ind, row in tqdm(testdf.iterrows()):
    
    lid = [l.upper() for l in row["Hinglish_lid"]]
        
    try:
        score = spavg_test(' '.join(lid))
    
    except IndexError as e:
        print(lid,e)
        score = np.nan
    
    sp_scores.append(score)
    
burstiness_scores = []

exceptionCount, IndexErrorCount, bothEnHiPresent = 0,0,0

for ind, row in tqdm(testdf.iterrows()):
    
    lid = [l.upper() for l in row["Hinglish_lid"]]
        
    try:
        score = burstiness(' '.join(lid))
    
    except IndexError as e:
        # print(lid,e)
        score = np.nan
        IndexErrorCount +=1
        
    except Exception as e:
        # print(lid, e)
        score = np.nan
        c = Counter(lid)
        if 'EN' in c and 'HI' in c:
            bothEnHiPresent+=1
        exceptionCount +=1
    
    burstiness_scores.append(score)

print(exceptionCount, bothEnHiPresent, IndexErrorCount)

testdf['Hinglish_burstiness_scores'] = burstiness_scores
testdf['Hinglish_sp_scores'] = sp_scores


class CodeMIxSentence():
    
    def __init__(self, sentence = None,
                         tokens = None,
                         LID_Tags = None,
                         PoS_Tags = None):
        
        self.sentence = sentence
        self.tokens = tokens
        self.LID_Tags = LID_Tags
        self.PoS_Tags = PoS_Tags
        self.length = len(LID_Tags) # has to changed to take length from number of token
        
        self.LID_count_map = dict(Counter(LID_Tags).most_common())
        self.PoS_count_map = dict(Counter(PoS_Tags).most_common())      
        
        lid_pos_combined = []
        
        for lid,pos in zip(self.LID_Tags, self.PoS_Tags):
            
            lid_pos_combined.append(pos+'_'+lid)
            
        self.LID_POS_count_map = dict(Counter(lid_pos_combined).most_common())


class SyMCoM:
    
    def __init__(self, 
                LID_tagset = None,
                PoS_tagset = None, 
                L1 = None, 
                L2 = None):
        
        self.LID_tagset = LID_tagset
        self.PoS_tagset = PoS_tagset
        self.l1 = L1
        self.l2 = L2
        
        
    def symcom_pos_tags(self, codemixsent_obj):
        
        symcom_scores_pos_tags = {}
        
        l1 = self.l1
        l2 = self.l2
        

            
                
        for postag in codemixsent_obj.PoS_count_map:
            
            if postag not in ['PUNCT','SYM', 'X']:
            
                pos_l1 = codemixsent_obj.LID_POS_count_map[postag+'_'+self.l1] if postag+'_'+self.l1 in codemixsent_obj.LID_POS_count_map else 0
                pos_l2 = codemixsent_obj.LID_POS_count_map[postag+'_'+self.l2] if postag+'_'+self.l2 in codemixsent_obj.LID_POS_count_map else 0
                
                try:
                    symcom_scores_pos_tags[postag+'_symcom'] = (pos_l1 - pos_l2) / (pos_l1 + pos_l2) 
                    
                except ZeroDivisionError: # to handle cases where there the LID for pos tag is not en or hi. 
                    pass
        return symcom_scores_pos_tags
    
    
        
        
    def symcom_sentence(self, codemixsent_obj):
        
        symcom_sentence = 0
        
        symcom_scores_pos_tags = self.symcom_pos_tags(codemixsent_obj)
        
        for pos, score in symcom_scores_pos_tags.items():
            pos = pos.split('_')[0]
            symcom_sentence += abs(score) * (codemixsent_obj.PoS_count_map[pos] / codemixsent_obj.length)     
            
        return symcom_sentence
    
symcom = SyMCoM(L1 = 'en',
                L2 = 'hi', 
                LID_tagset = ['hi', 'en', 'ne', 'univ', 'acro'],
                PoS_tagset = ['NOUN', 'ADV', 'VERB', 'AUX', 'ADJ', 'ADP', 'PUNCT', 'DET', 'PRON', 'PROPN', 'PART', 'CCONJ', 'SCONJ', 'INTJ', 'NUM', 'SYM','X'])


symcom = SyMCoM(L1 = 'en',
                L2 = 'hi', 
                LID_tagset = ['hi', 'en', 'ne', 'univ', 'acro'],
                PoS_tagset = ['NOUN', 'ADV', 'VERB', 'AUX', 'ADJ', 'ADP', 'PUNCT', 'DET', 'PRON', 'PROPN', 'PART', 'CCONJ', 'SCONJ', 'INTJ', 'NUM', 'SYM','X'])


symcom_scores_pos, symcom_sent = [],[]

div_by_zero_error_count, errors  = 0 , []

for ind, row in tqdm(testdf.iterrows()):

    cm_sentence = CodeMIxSentence(sentence = None,
                                     tokens = row['Hinglish_norm'],
                                     LID_Tags = row['Hinglish_lid'],
                                     PoS_Tags = row['Hinglish_pos'])
    symcom_scores_pos.append(symcom.symcom_pos_tags(cm_sentence))
    symcom_sent.append(symcom.symcom_sentence(cm_sentence))
    
    
testdf['symcom_scores_pos'] = symcom_scores_pos
testdf['symcom_scores_sent'] = symcom_sent

df2 = pd.json_normalize(testdf["symcom_scores_pos"])
testdf = pd.concat([testdf,df2], axis = 1)
print(testdf.shape)
testdf.reset_index(inplace = True)

testdf.shape


def count_plusone(sample):
    count = 0
    try:
        assert isinstance(sample,dict)
    except AssertionError:
        return np.nan
    for k,v in sample.items():
        if v == 1:
            count+=1
    return count

def count_minus1(sample):
    print(sample)
    count = 0
    for k,v in sample.items():
        if v == -1:
            count+=1
    return count

def countNaN(sample):
    count = 14 - len(sample.keys()) 
    return count

testdf["symcom_+1_count"] = testdf["symcom_scores_pos"].apply(count_plusone)
testdf["symcom_-1_count"] = testdf["symcom_scores_pos"].apply(count_minus1)
testdf["symcom_na_count"] = testdf["symcom_scores_pos"].apply(countNaN)
testdf["symcom_others"] = testdf.apply(lambda row : 14 - row["symcom_+1_count"] - row["symcom_-1_count"]- row["symcom_na_count"] , axis = 1)    


791it [00:00, 12729.27it/s]
791it [00:00, 15841.47it/s]
791it [00:00, 5908.15it/s]


72 12 0


791it [00:00, 8554.93it/s]


(791, 32)
{'NOUN_symcom': 0.3333333333333333, 'PART_symcom': -1.0, 'PRON_symcom': -1.0, 'ADP_symcom': -1.0, 'ADJ_symcom': 1.0, 'ADV_symcom': -1.0, 'VERB_symcom': -1.0}
{'ADJ_symcom': -0.6, 'NOUN_symcom': -0.6, 'ADP_symcom': -1.0, 'ADV_symcom': -1.0, 'AUX_symcom': -1.0, 'PRON_symcom': -1.0, 'VERB_symcom': -1.0, 'CCONJ_symcom': -1.0}
{'NOUN_symcom': -1.0, 'ADP_symcom': -1.0, 'PRON_symcom': -1.0, 'VERB_symcom': -1.0, 'AUX_symcom': -1.0, 'CCONJ_symcom': -1.0, 'SCONJ_symcom': -1.0, 'DET_symcom': -1.0, 'PART_symcom': -1.0, 'ADV_symcom': -1.0, 'ADJ_symcom': -1.0}
{'NOUN_symcom': 0.7777777777777778, 'ADP_symcom': -1.0, 'ADV_symcom': -1.0, 'VERB_symcom': -1.0, 'ADJ_symcom': 1.0, 'CCONJ_symcom': -1.0, 'SCONJ_symcom': -1.0, 'PRON_symcom': -1.0, 'AUX_symcom': -1.0}
{'ADP_symcom': -1.0, 'AUX_symcom': -1.0, 'NOUN_symcom': 0.5, 'VERB_symcom': -1.0, 'DET_symcom': -1.0, 'SCONJ_symcom': -1.0, 'PART_symcom': -1.0, 'PRON_symcom': -1.0, 'ADJ_symcom': -1.0, 'ADV_symcom': -1.0}
{'NOUN_symcom': 0.5, 'PROPN_sy

# Minicons scoring

In [11]:
from minicons import scorer
import torch
mlm_model = scorer.MaskedLMScorer('xlm-roberta-large', 'cuda')





tokens_strings = [" ".join(sample) for sample in testdf["Hinglish_norm"]]


results = []
for ind, batch in tqdm(enumerate(tokens_strings)):
    
    # print(len(batch))
    try:
        score = mlm_model.sequence_score(batch, reduction = lambda x: -x.sum(0).item())

        results.extend(score)
        
    except RuntimeError:
               
        results.extend([np.nan])
        
    torch.cuda.empty_cache()

    
count= 0

for el in tqdm(results):
    if el != el:
        count+=1
        
print(f"Num of nans in the xlmr results : {count}")

testdf["Hingilish_xlmr_scores"] = results


tokens_strings = [[" ".join(sent) for sent in sample] for sample in testdf["hum_gen_norm"]]

results = []
for ind, batch in tqdm(enumerate(tokens_strings)):

    rowscores = []
    
    for ind1, sent in enumerate(batch):
        
    
        # print(len(batch))
        try:
            score = mlm_model.sequence_score(sent, reduction = lambda x: -x.sum(0).item())

            rowscores.extend(score)

        except RuntimeError:

            rowscores.extend([np.nan])
            
    results.append(rowscores)

    torch.cuda.empty_cache()
        
    

    
count= 0

for el in tqdm(results):
    
    if el != el:
        count+=1
        
print(f"Num of nans in the xlmr results : {count}")


testdf["hum_gen_xlmr_scores"] = results

Downloading:   0%|          | 0.00/616 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.09G [00:00<?, ?B/s]

791it [01:30,  8.70it/s]
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 791/791 [00:00<00:00, 2243201.13it/s]


Num of nans in the xlmr results : 56


791it [03:05,  4.26it/s]
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 791/791 [00:00<00:00, 1972469.95it/s]

Num of nans in the xlmr results : 0


In [14]:
for ind, row in testdf.iterrows():
    for col in testdf.columns:
        print(f"{col} : {row[col]}")
        
    break

index : 0
English : Late Puranchandra Gupta never allowed any power to force him into restricting his freedom of press.

Hindi : स्वर्गीय पूरनचन्द्र गुप्ता कभी भी अपने समाचारपत्र साम्न्त बनने की अपनी इच्छाशक्ति से नहीं डगमगाए।

Hinglish : late puranchandra gupta kabhi bhi apne press samnt power ki apni ichchhashakti se nahin dagamgae.

hum_gen : ['Late Puranchandra Gupta never allowed apne samacharpatra samant banne ki ichchhasakti se.\n', 'Swargiy Puranchandra Gupta kabhi bhi apne restricting his freedom of press se nahi dagmagaye.\n']
Hinglish_csnliop : [['late', 'late', 'en'], ['puranchandra', 'puranchandra', 'ne'], ['gupta', 'gupta', 'ne'], ['kabhi', 'कभी', 'hi'], ['bhi', 'भी', 'hi'], ['apne', 'अपने', 'hi'], ['press', 'press', 'en'], ['samnt', 'samnt', 'ne'], ['power', 'power', 'en'], ['ki', 'की', 'hi'], ['apni', 'अपनी', 'hi'], ['ichchhashakti', 'इच्छाशक्ति', 'hi'], ['se', 'से', 'hi'], ['nahin', 'नहीं', 'hi'], ['dagamgae.', 'डगमगाया.', 'hi']]
hum_gen_csnliop : [[['Late', 'Late', 'e

In [15]:
testdf.to_json(testdf_file_path)

# xlm-r features

In [17]:
import pandas as pd
import torch
from tqdm import tqdm 

xlmr = torch.hub.load('pytorch/fairseq', 'xlmr.large').cuda()
xlmr.eval()



Downloading: "https://github.com/pytorch/fairseq/archive/main.zip" to /home/prashantk/.cache/torch/hub/main.zip


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


fatal: Not a git repository (or any parent up to mount point /home)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
running build_ext
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingfac

2022-05-08 22:34:03 | INFO | fairseq.file_utils | http://dl.fbaipublicfiles.com/fairseq/models/xlmr.large.tar.gz not found in cache, downloading to /tmp/tmpvolkkwsj
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1028340964/1028340964 [01:04<00:00, 15857432.14B/s]
2022-05-08 22:35:08 | INFO | fairseq.file_utils | copying /tmp/tmpvolkkwsj to cache at /home/prashantk/.cache/torch/pytorch_fairseq/3f864e15bb396f062dd37494309dbc4238416edd1f8ef293df18b1424813f2fe.cf46c7deb6b9eaa3e47c17b9fc181669c52bc639c165fbc69166a61487662ac9
2022-05-08 22:35:18 | INFO | fairseq.file_utils | creating metadata file for /home/prashantk/.cache/torch/pytorch_fairseq/3f864e15bb396f062dd37494309dbc4238416edd1f8ef293df18b1424813f2fe.cf46c7deb6b9eaa3e47c17b9fc181669c52bc639c165fbc69166a61487662ac9
2022-05-08 22:35:18 | INFO | fairseq.file_utils | removing temp file /tmp/tmpvolkkwsj
2022-05-08 22:35:18 | I

RobertaHubInterface(
  (model): RobertaModel(
    (encoder): RobertaEncoder(
      (sentence_encoder): TransformerEncoder(
        (dropout_module): FairseqDropout()
        (embed_tokens): Embedding(250002, 1024, padding_idx=1)
        (embed_positions): LearnedPositionalEmbedding(514, 1024, padding_idx=1)
        (layernorm_embedding): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (layers): ModuleList(
          (0): TransformerEncoderLayerBase(
            (self_attn): MultiheadAttention(
              (dropout_module): FairseqDropout()
              (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
            )
            (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (d

In [19]:


testdf = pd.read_json(testdf_file_path)

def returnSynthSentences(df):
    csnliHinglish = list(df["Hinglish_csnliop"])
    sentences = []
    for line in csnliHinglish:
        sent = []
        for token in line:
            sent.append(token[1])
        sent = " ".join(sent)
        sentences.append(sent)
    
    return sentences

def returnHumSentences(df):
    csnliHinglish = list(df["hum_gen_csnliop"])
    sentences = []
    for sents in csnliHinglish:
        ref = []
        for line in sents:
            sent = []
            for token in line:
                sent.append(token[1])
            sent = " ".join(sent)
            ref.append(sent)
            
        sentences.append(ref)
    
    return sentences

testSynth = returnSynthSentences(testdf)

testHuman = returnHumSentences(testdf)


print(len(testSynth), len(testHuman))



def returnSentenceVectors(sents):
    with torch.no_grad():
        vectors = []
        for line in tqdm(sents):
            if isinstance(line, list):
                refs = []
                for st in line:
                    tokens = xlmr.encode(st.strip())
                    llf = xlmr.extract_features(tokens)
                    llf = llf.mean(dim=1)
                    refs.append(llf)
                refs = torch.cat(refs, dim=0)
                refs = refs.mean(dim=0).unsqueeze(0)
                vectors.append(refs)

            else:
                tokens = xlmr.encode(line)
                llf = xlmr.extract_features(tokens)
                llf = llf.mean(dim=1)
                vectors.append(llf)

        vectors = torch.cat(vectors, dim=0)
        return vectors
    
    
testSynthVecs = returnSentenceVectors(testSynth)
testHumanVecs = returnSentenceVectors(testHuman)

torch.save(testSynthVecs, "test.synth.pt")
torch.save(testHumanVecs, "test.human.pt")

791 791


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 791/791 [00:32<00:00, 24.39it/s]
